### Meeting a Sayed Athar's request, I'm using the Kernel altered by Khoi Nguyen to explain how the whole code works.
### If any part is not clear, please comment.  
### Please upvote if it was helpful.

In [1]:
import pandas as pd
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting
import numpy.fft as fft
import matplotlib.pyplot as plt
import pywt

Using TensorFlow backend.


In [2]:
# select how many folds will be created
N_SPLITS = 5
# it is just a constant with the measurements data size
sample_size = 800000

In [3]:
# It is the official metric used in this competition
# below is the declaration of a function used inside the keras model, calculation with K (keras backend / thensorflow)
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [4]:
y_true = K.variable([[0,0,0],[1,1,1],[1,0,1]])
y_pred = K.variable([[1,0,0],[0,0,1],[1,1,1]])
y_true2 = K.variable([0,0,0,1,1,1,1,0,1])
y_pred2 = K.variable([1,0,0,0,0,1,1,1,1])

print(K.eval(matthews_correlation(y_true, y_pred)),K.eval(matthews_correlation(y_true2, y_pred2)))

0.1 0.1


In [5]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [6]:
# just load train data
df_train = pd.read_csv('../input/metadata_train.csv')
# set index, it makes the data access much faster
df_train = df_train.set_index(['id_measurement', 'phase'])
df_train.head()

signal_id  target
id_measurement phase                   
0              0              0       0
               1              1       0
               2              2       0
1              0              3       1
               1              4       1

In [7]:
df_train.tail()

signal_id  target
id_measurement phase                   
2902           1           8707       0
               2           8708       0
2903           0           8709       0
               1           8710       0
               2           8711       0

In [8]:
# in other notebook I have extracted the min and max values from the train data, the measurements
max_num = 128

In [9]:
n_dim=160
w = pywt.Wavelet('coif5')
nl=5

In [10]:
# This is one of the most important peace of code of this Kernel
# Any power line contain 3 phases of 800000 measurements, or 2.4 millions data 
# It would be praticaly impossible to build a NN with an input of that size
# The ideia here is to reduce it each phase to a matrix of <n_dim> bins by n features
# Each bean is a set of 5000 measurements (800000 / 160), so the features are extracted from this 5000 chunk data.
def transform_ts(ts, n_dim=n_dim, min_max=(-1,1)):
    # convert data into -1 to 1
    # ts /= 128.0
    # bucket or chunk size, 5000 in this case (800000 / 160)
    bucket_size = int(sample_size / n_dim)
    # new_ts will be the container of the new data
    new_ts = []
    # this for iteract any chunk/bucket until reach the whole sample_size (800000)
    
    '''some of features I added'''
#     plt.plot(ts)
#     plt.plot(fzfit)
#     new_ts = np.append(new_ts,[phi])
#     new_ts = np.append(new_ts,percentil_fz)
    
    #print(new_ts)
    
    for i in range(0, sample_size, bucket_size):
        # cut each bucket to ts_range
        
        ts_range = ts[i:i + bucket_size]
        cwt = pywt.wavedec(ts_range, w, level=nl)
        # calculate each feature
        nj=[]
        for j in range(nl+1):
            mean = cwt[j].mean()
            std = cwt[j].std()
            percentil = np.percentile(cwt[j], [0, 1, 25, 50, 75, 99, 100]) 
            nj += [mean,std]
            nj += list(percentil)
        new_ts += [nj]
    #print(np.asarray(new_ts).shape)
    return np.asarray(new_ts)

In [11]:
col="3"
ts=pq.read_pandas('../input/train.parquet', columns=[col]).to_pandas()[col]
transform_ts(ts, n_dim=n_dim, min_max=(-1,1))

array([[ -88.53084818,    2.32666522,  -94.73027722, ...,    0.27988937,
           0.97479176,    3.54929324],
       [ -83.39593658,    2.11978358,  -89.6592986 , ...,    0.31641296,
           1.21128039,   10.03361081],
       [ -78.37265849,    2.41833206,  -82.37566106, ...,    0.30436853,
           0.96117711,    4.83131757],
       ...,
       [-100.3101889 ,    1.69457386, -103.99902433, ...,    0.28730831,
           1.0557028 ,    1.95649691],
       [ -96.94308645,    2.20236516, -102.53004156, ...,    0.28320539,
           1.00724049,    5.10392475],
       [ -91.97224005,    2.88215366,  -97.11812291, ...,    0.30026385,
           1.09877933,    9.16013838]])

In [12]:
# this function take a piece of data and convert using transform_ts(), but it does to each of the 3 phases
# if we would try to do in one time, could exceed the RAM Memmory
def prep_data(df,IDs,path):
    start, end = IDs[0], IDs[-1]
    # load a piece of data from file
    praq = pq.read_pandas(path, columns=[str(i) for i in range(start*3, end*3)]).to_pandas()
    X = []
    y = []
    
    for ID in range(start, end):
        X_signal=[]
        y_signal=[]
        # for each phase of the signal
        for phase in [0,1,2]:
            # extract from df_train both signal_id and target to compose the new data sets
            signal_id, target = df.loc[ID].loc[phase]
            # extract and transform data into sets of features
            X_signal.append(transform_ts(praq[str(signal_id)]))
            y_signal.append(target)
        # concatenate all the 3 phases in one matrix
        X_signal=np.concatenate(X_signal,axis=1)
        X.append(X_signal)
        y.append(y_signal)
    X = np.asarray(X)
    y = np.asarray(y)
    print(X.shape,y.shape)
    return X, y

In [13]:
# this code is very simple, divide the total size of the df_train into two sets and process it

def load_all(df,path,n_slit):
    X = []
    y = []
    ID_list = []
    start = df.index.levels[0][0]
    end = df.index.levels[0][-1]+1
    print(start,end)
    Nperpart=int((end-start)/n_slit)
    
    while start+Nperpart < end:
        ID_list.append([start,start+Nperpart])
        start += Nperpart
    ID_list.append([start,end])
    
    for IDs in tqdm(ID_list):
        print(IDs)
        X_temp, y_temp = prep_data(df,IDs,path)
        X.append(X_temp)
        y.append(y_temp)

    X = np.concatenate(X,axis=0)
    y = np.concatenate(y,axis=0)
    return X,y




In [14]:
%%time
X,y = load_all(df_train,'../input/train.parquet',3)
print(X.shape, y.shape)

  0%|          | 0/3 [00:00<?, ?it/s]

0 2904
[0, 968]


 33%|███▎      | 1/3 [15:25<30:50, 925.27s/it]

(968, 160, 162) (968, 3)
[968, 1936]


 67%|██████▋   | 2/3 [30:44<15:23, 923.33s/it]

(968, 160, 162) (968, 3)
[1936, 2904]


100%|██████████| 3/3 [46:18<00:00, 926.77s/it]

(968, 160, 162) (968, 3)


(2904, 160, 162) (2904, 3)
CPU times: user 46min 7s, sys: 10.7 s, total: 46min 18s
Wall time: 46min 19s


In [15]:
# This is NN LSTM Model creation
def model_lstm(input_shape):
    # The shape was explained above, must have this order
    inp = Input(shape=(input_shape[1], input_shape[2],))
    # This is the LSTM layer
    # Bidirecional implies that the 160 chunks are calculated in both ways, 0 to 159 and 159 to zero
    # although it appear that just 0 to 159 way matter, I have tested with and without, and tha later worked best
    # 128 and 64 are the number of cells used, too many can overfit and too few can underfit
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(inp)
    # The second LSTM can give more fire power to the model, but can overfit it too
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    # Attention is a new tecnology that can be applyed to a Recurrent NN to give more meanings to a signal found in the middle
    # of the data, it helps more in longs chains of data. A normal RNN give all the responsibility of detect the signal
    # to the last cell. Google RNN Attention for more information :)
    x = Attention(input_shape[1])(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64, activation="relu")(x)
    # x = Dense(64, activation="relu")(x)
    # A binnary classification as this must finish with shape (1,)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    # Pay attention in the addition of matthews_correlation metric in the compilation, it is a success factor key
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    #model.summary()
    return model

In [16]:
# Here is where the training happens
# First, create a set of indexes of the 5 folds
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, np.zeros(shape=(X.shape[0], 1))))
preds_val = []
y_val = []
cal_list=[]


In [17]:
%%time

for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    cal_list.append(val_idx)
    # use the indexes to extract the folds in the train and validation data
    train_X, train_y, val_X, val_y = X[train_idx], y[train_idx], X[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit(train_X, train_y, batch_size=128, epochs=50, validation_data=[val_X, val_y], callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict(val_X, batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)
y_val = np.concatenate(y_val)
cal_list=np.concatenate(cal_list)
print(preds_val.shape, y_val.shape)

Beginning fold 1
Train on 2323 samples, validate on 581 samples
Epoch 1/50
2323/2323 [==============================] - 4s 2ms/step - loss: 0.2908 - matthews_correlation: 0.0019 - val_loss: 0.2527 - val_matthews_correlation: 0.0000e+00

Epoch 00001: val_matthews_correlation improved from -inf to 0.00000, saving model to weights_0.h5
Epoch 2/50
2323/2323 [==============================] - 1s 594us/step - loss: 0.2048 - matthews_correlation: 0.0000e+00 - val_loss: 0.2295 - val_matthews_correlation: 0.0000e+00

Epoch 00002: val_matthews_correlation did not improve from 0.00000
Epoch 3/50
2323/2323 [==============================] - 1s 599us/step - loss: 0.1777 - matthews_correlation: 0.1486 - val_loss: 0.1781 - val_matthews_correlation: 0.3680

Epoch 00003: val_matthews_correlation improved from 0.00000 to 0.36799, saving model to weights_0.h5
Epoch 4/50
2323/2323 [==============================] - 1s 593us/step - loss: 0.1439 - matthews_correlation: 0.2288 - val_loss: 0.1387 - val_matthe

In [18]:
# The output of this kernel must be binary (0 or 1), but the output of the NN Model is float (0 to 1).
# So, find the best threshold to convert float to binary is crucial to the result
# this piece of code is a function that evaluates all the possible thresholds from 0 to 1 by 0.01

def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        score = K.eval(matthews_correlation(K.variable(y_true), K.variable((y_proba > threshold).astype(np.int))))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    print(best_threshold,best_score)
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    
    return search_result

In [19]:
best_threshold = threshold_search(y_val,preds_val)['threshold']

100%|██████████| 100/100 [00:54<00:00,  1.10it/s]

0.49 0.7293547


In [20]:
wrongpred = []
preds_val = (preds_val > best_threshold).astype(np.int)
for i,y in enumerate(y_val):
    if (preds_val[i]!= y).any():
        print(cal_list[i],preds_val[i],y)
        wrongpred.append(cal_list[i])
print(wrongpred)
    

13 [1 1 1] [0 0 0]
76 [0 0 0] [1 1 1]
88 [1 1 1] [0 0 0]
235 [0 0 0] [1 1 1]
271 [0 0 0] [1 0 0]
276 [1 1 1] [0 0 0]
443 [1 1 1] [0 0 1]
620 [1 1 1] [1 0 1]
695 [1 0 1] [1 1 1]
699 [0 0 0] [1 1 1]
962 [1 0 1] [0 0 0]
988 [1 1 1] [0 1 1]
1010 [0 0 0] [1 1 1]
1076 [1 1 1] [1 1 0]
1081 [0 0 0] [1 1 1]
1132 [1 1 1] [1 0 0]
1310 [1 1 1] [0 0 0]
1537 [1 1 1] [1 0 1]
1680 [1 1 0] [0 0 0]
1810 [1 1 1] [0 0 0]
1855 [1 1 1] [0 0 0]
1884 [1 1 1] [0 0 1]
1897 [1 1 1] [0 0 0]
1899 [1 1 1] [0 0 1]
1965 [1 1 1] [0 0 0]
2090 [1 1 1] [0 0 0]
2126 [1 0 1] [1 1 1]
2212 [0 0 0] [1 1 1]
2623 [1 1 1] [1 1 0]
2688 [1 1 1] [0 0 0]
2702 [1 1 1] [0 0 0]
2876 [0 0 0] [0 0 1]
41 [1 1 1] [0 0 0]
90 [0 0 0] [1 1 1]
96 [1 1 1] [0 0 1]
145 [0 0 0] [1 1 1]
608 [0 0 0] [1 0 1]
706 [1 1 1] [1 1 0]
809 [1 0 1] [0 0 0]
894 [1 1 1] [1 0 0]
900 [1 1 1] [0 0 0]
923 [1 1 1] [0 0 0]
1055 [1 1 0] [0 0 0]
1103 [0 0 0] [1 1 1]
1256 [1 1 1] [1 0 1]
1277 [1 0 1] [1 0 0]
1442 [1 1 1] [0 0 0]
1476 [1 1 1] [0 0 0]
1549 [1 1 1] [0 0 0]

In [21]:
%%time
# Now load the test data
# This first part is the meta data, not the main data, the measurements
meta_test = pd.read_csv('../input/metadata_test.csv')
meta_test = meta_test.set_index(['id_measurement', 'phase'])
meta_test.head()

CPU times: user 32 ms, sys: 8 ms, total: 40 ms
Wall time: 26.6 ms


In [22]:
meta_test['target']=0

In [23]:
meta_test.head()

signal_id  target
id_measurement phase                   
2904           0           8712       0
               1           8713       0
               2           8714       0
2905           0           8715       0
               1           8716       0

In [24]:
meta_test.tail()

signal_id  target
id_measurement phase                   
9681           1          29044       0
               2          29045       0
9682           0          29046       0
               1          29047       0
               2          29048       0

In [25]:
%%time
X_test,y_test = load_all(meta_test,'../input/test.parquet',10)
print(X_test.shape)

  0%|          | 0/11 [00:00<?, ?it/s]

2904 9683
[2904, 3581]


  9%|▉         | 1/11 [10:57<1:49:35, 657.60s/it]

(677, 160, 162) (677, 3)
[3581, 4258]


 18%|█▊        | 2/11 [21:49<1:38:23, 655.96s/it]

...

 91%|█████████ | 10/11 [1:48:15<10:48, 648.21s/it]

(677, 160, 162) (677, 3)
[9674, 9683]


100%|██████████| 11/11 [1:48:25<00:00, 456.61s/it]

(9, 160, 162) (9, 3)


(6779, 160, 162)
CPU times: user 1h 47min 59s, sys: 24.8 s, total: 1h 48min 24s
Wall time: 1h 48min 26s


In [26]:
submission = pd.read_csv('../input/sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [27]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict(X_test, batch_size=300, verbose=1)
    print(pred.shape)
    preds_test.append(pred)
print(np.asarray(preds_test).shape)

6779/6779 [==============================] - 3s 442us/step
(6779, 3)
6779/6779 [==============================] - 3s 451us/step
(6779, 3)
6779/6779 [==============================] - 3s 449us/step
(6779, 3)
6779/6779 [==============================] - 3s 447us/step
(6779, 3)
6779/6779 [==============================] - 3s 449us/step
(6779, 3)
(5, 6779, 3)


In [28]:
preds_test = (np.mean(preds_test, axis=0).reshape(-1) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [29]:
print(preds_test[:99])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [30]:
submission['target'] = preds_test
submission.to_csv('submission.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0
